# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 12:14PM,250808,21,633538,"NBTY Global, Inc.",Released
1,Nov 15 2022 12:09PM,250807,19,MSP215782,"Methapharm, Inc.",Executing
2,Nov 15 2022 12:09PM,250807,19,MSP215795,"Methapharm, Inc.",Executing
3,Nov 15 2022 12:09PM,250807,19,MSP215813,"Methapharm, Inc.",Executing
4,Nov 15 2022 12:09PM,250807,19,MSP215814,"Methapharm, Inc.",Executing
5,Nov 15 2022 12:09PM,250807,19,MSP215840,"Methapharm, Inc.",Executing
6,Nov 15 2022 12:09PM,250807,19,MSP215843,"Methapharm, Inc.",Executing
7,Nov 15 2022 12:09PM,250807,19,MSP215845,"Methapharm, Inc.",Executing
8,Nov 15 2022 12:09PM,250807,19,MSP215855,"Methapharm, Inc.",Executing
9,Nov 15 2022 12:09PM,250807,19,MSP215862,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,250805,Released,1
29,250806,Released,1
30,250807,Executing,8
31,250807,Released,2
32,250808,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250804,NaN,NaN,1.0
250805,NaN,NaN,1.0
250806,NaN,NaN,1.0
250807,NaN,8.0,2.0
250808,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250703,23.0,1.0,0.0
250755,0.0,7.0,22.0
250757,0.0,0.0,1.0
250766,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250703,23,1,0
250755,0,7,22
250757,0,0,1
250766,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250703,23,1,0
2,250755,0,7,22
3,250757,0,0,1
4,250766,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250703,23,1,
2,250755,,7,22
3,250757,,,1
4,250766,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc."
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc."
11,Nov 15 2022 12:08PM,250806,10,Emerginnova
12,Nov 15 2022 12:07PM,250805,15,Lonza
13,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc."
25,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc."
26,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC
30,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc."
31,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation
52,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc.",,,1
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",,8,2
2,Nov 15 2022 12:08PM,250806,10,Emerginnova,,,1
3,Nov 15 2022 12:07PM,250805,15,Lonza,,,1
4,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",,,12
5,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",,,1
6,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC,,,4
7,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc.",,,1
8,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation,,,21
9,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation,,,18


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc.",1,,
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",2,8,
2,Nov 15 2022 12:08PM,250806,10,Emerginnova,1,,
3,Nov 15 2022 12:07PM,250805,15,Lonza,1,,
4,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12,,
5,Nov 15 2022 12:05PM,250804,19,"GUSA Granules USA, Inc.",1,,
6,Nov 15 2022 12:05PM,250803,10,MedStone Pharma LLC,4,,
7,Nov 15 2022 12:00PM,250801,19,"GUSA Granules USA, Inc.",1,,
8,Nov 15 2022 11:43AM,250800,10,ISDIN Corporation,21,,
9,Nov 15 2022 11:38AM,250787,10,ISDIN Corporation,18,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc.",1,,
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",2,8,
2,Nov 15 2022 12:08PM,250806,10,Emerginnova,1,,
3,Nov 15 2022 12:07PM,250805,15,Lonza,1,,
4,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",2.0,8.0,NaN
2,Nov 15 2022 12:08PM,250806,10,Emerginnova,1.0,NaN,NaN
3,Nov 15 2022 12:07PM,250805,15,Lonza,1.0,NaN,NaN
4,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 12:14PM,250808,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Nov 15 2022 12:09PM,250807,19,"Methapharm, Inc.",2.0,8.0,0.0
2,Nov 15 2022 12:08PM,250806,10,Emerginnova,1.0,0.0,0.0
3,Nov 15 2022 12:07PM,250805,15,Lonza,1.0,0.0,0.0
4,Nov 15 2022 12:07PM,250802,20,"Exact-Rx, Inc.",12.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2507895,77.0,17.0,6.0
12,501573,2.0,0.0,0.0
15,752243,90.0,18.0,0.0
16,250766,1.0,0.0,0.0
19,1253903,5.0,9.0,23.0
20,501573,17.0,0.0,4.0
21,1003152,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2507895,77.0,17.0,6.0
1,12,501573,2.0,0.0,0.0
2,15,752243,90.0,18.0,0.0
3,16,250766,1.0,0.0,0.0
4,19,1253903,5.0,9.0,23.0
5,20,501573,17.0,0.0,4.0
6,21,1003152,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,17.0,6.0
1,12,2.0,0.0,0.0
2,15,90.0,18.0,0.0
3,16,1.0,0.0,0.0
4,19,5.0,9.0,23.0
5,20,17.0,0.0,4.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,12,Released,2.0
2,15,Released,90.0
3,16,Released,1.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,6.0,0.0,0.0,0.0,23.0,4.0,0.0
Executing,17.0,0.0,18.0,0.0,9.0,0.0,0.0
Released,77.0,2.0,90.0,1.0,5.0,17.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,23.0,4.0,0.0
1,Executing,17.0,0.0,18.0,0.0,9.0,0.0,0.0
2,Released,77.0,2.0,90.0,1.0,5.0,17.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,6.0,0.0,0.0,0.0,23.0,4.0,0.0
1,Executing,17.0,0.0,18.0,0.0,9.0,0.0,0.0
2,Released,77.0,2.0,90.0,1.0,5.0,17.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()